In [1]:
from torch.utils.data import DataLoader
from model import build_transformer
from utils import generate_causal_mask, greedy_decode, calculate_bleu_score
from configs.overfit import Config
from transformers import AutoTokenizer
from data import WMTDataset
import torch

tokenizer = AutoTokenizer.from_pretrained("facebook/wmt19-en-de")
special_token_dict = {"bos_token": "<s>"}
tokenizer.add_special_tokens(special_token_dict)

tokenizer.encode("hi my name is neet")
    
ds = WMTDataset("overfit.csv", tokenizer, tokenizer, 200)

config = Config()

model = build_transformer(config)

state_dict = torch.load("overfit.pth")['model_state_dict']

model.load_state_dict(state_dict=state_dict)

dataloader = DataLoader(ds, batch_size=1, shuffle=False)

loss = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.convert_tokens_to_ids("<pad>"), label_smoothing=0.1)



c:\Users\neetm\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
for batch in dataloader:
    src, tgt, label, src_mask, tgt_mask = batch
    enc_self_attn_mask = src_mask.unsqueeze(2) | src_mask.unsqueeze(3)
    causal_mask = generate_causal_mask(config.seq_len)
    dec_self_attn_mask = tgt_mask.unsqueeze(2) | tgt_mask.unsqueeze(3) | causal_mask
    dec_cross_attn_mask = tgt_mask.unsqueeze(3) | src_mask.unsqueeze(2)
    
    y = model(src, tgt, enc_self_attn_mask, dec_self_attn_mask, dec_cross_attn_mask)
    
    
    
    break

In [4]:
_, text = greedy_decode(src, src_mask, model, tokenizer, config, 'cpu')

In [5]:
tokenizer.decode(label[0].tolist())

'Von daher werden sie gegen ihren Ex-Coach sicher ganz besonders motiviert sein. </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [9]:
tokenizer.bos_token

'<s>'

In [ ]:
import torchmetrics
metrics = torchmetrics.text.BLEUScore()
calculate_bleu_score(metrics, text, text)

1.0

In [ ]:
from torchmetrics.text import BLEUScore
preds = ['Von daher werden sie gegen ihren Ex-Coach sicher ganz besonders motiviert sein.']
target = [['Von daher werden sie gegen ihren Ex-Coach sicher ganz besonders motiviert sein.', 'Von daher werden sie gegen ihren Ex-Coach sicher ganz besonders motiviert sein.']]
bleu = BLEUScore()
bleu(preds, target)

tensor(1.)

In [ ]:
tokenizer.vocab_size


42024